# Library Usage in Seattle 2005-2020

## Data Cleaning

### Import required libraries

In [1]:
# standard dataframe packages
import pandas as pd
import numpy as np

# saving packages
import pickle
import gzip

### Load checkout data

Since the data set is so large, I'll specify only the columns that I want in the DataFrame. This will effectively drop the following columns:
- `ID`
- `CheckoutYear`
- `BibNumber`
- `ItemBarcode`
- `ItemType`
    
I want to note that the `ItemType` and `Collection` columns are very similar, but the code in the `Collection` column contains more information within the `category_group` column that I add onto the DataFrame using the `data_dictionary.csv` file (see below). More specifically, the `ItemType` code yields mostly "Miscellaneous" results, whereas the `Collection` code yields differentiates between "Fiction" and "Nonfiction", among others. This could be useful information later on, so I found it best to drop the `ItemType` column.

In [2]:
%%time

# columns to load
usecols = ['Collection', 'CallNumber', 'ItemTitle', 'Subjects', 'CheckoutDateTime']

# load data (first 10 million rows)
df = pd.read_csv('data/Checkouts_By_Title__Physical_Items_.csv', nrows=10000000, usecols=usecols)

# rename columns to my preferred format
df.columns = ['collection', 'call_number', 'title', 'subjects', 'date']

CPU times: user 24.2 s, sys: 1.47 s, total: 25.7 s
Wall time: 25.7 s


In [3]:
df.head()

,collection,call_number,title,subjects,date
0,nadvd,DVD FIREWAL,Firewall,"Kidnapping Drama, Video recordings for the hea...",02/13/2008 07:38:00 PM
1,nanf,793.2 C7744B 2001,best baby shower book a complete guide for par...,Showers Parties,07/23/2008 02:53:00 PM
2,nyfic,YA WESTERF,Uglies,"Fantasy, Teenage girls Fiction, Beauty Persona...",12/23/2009 04:20:00 PM
3,napar,618.4 L9511D 2009,doula guide to birth secrets every pregnant wo...,"Doulas, Childbirth",11/16/2010 12:04:00 PM
4,canf,641.692 M8216S 2005,Salmon a cookbook,Cookery Salmon,04/26/2009 01:29:00 PM


In [4]:
# check shape
df.shape

(10000000, 5)

In [5]:
# check for nan values
df.isna().sum()

collection          0
call_number      1123
title          105542
subjects       137301
date                0
dtype: int64

The most important columns (`collection` and `date`) have no NaN values.

In [6]:
# check datatypes
df.dtypes

collection     object
call_number    object
title          object
subjects       object
date           object
dtype: object

### Convert `date` column to datetime

In [7]:
# look at an example before conversion
df.loc[0, 'date']

'02/13/2008 07:38:00 PM'

In [8]:
# specify the format
dt_format = '%m/%d/%Y %I:%M:%S %p'

In [9]:
%%time

# convert to datetime, dropping the hour-minute-second stamp using the `dt.date` attribute
df['date'] = pd.to_datetime(df.date, format=dt_format).dt.date

# confirm it worked
df.loc[0, 'date']

CPU times: user 33.8 s, sys: 199 ms, total: 34 s
Wall time: 34 s


datetime.date(2008, 2, 13)

### Load other info from data dictionary

In [10]:
# load data
dd = pd.read_csv('data/data_dictionary.csv')

# rename columns to my preferred format
dd.columns = ['code', 'description', 'code_type', 'format_group', 'format_subgroup', 
              'category_group', 'category_subgroup', 'age_group']

# take a look
dd.head()

,code,description,code_type,format_group,format_subgroup,category_group,category_subgroup,age_group
0,cazover,CA7-zine collection oversize,ItemCollection,Print,Book,Periodical,NaN,Adult
1,caziner,CA7-zine collection reference,ItemCollection,Print,Book,Periodical,NaN,Adult
2,cazval,CA7-zine collection valuable mat.,ItemCollection,Print,Book,Periodical,NaN,Adult
3,nga,Northgate Branch,Location,NaN,NaN,NaN,NaN,NaN
4,hip,High Point Branch,Location,NaN,NaN,NaN,NaN,NaN


In [11]:
# check shape
dd.shape

(580, 8)

In [12]:
# check for nan values
dd.isna().sum()

code                   0
description            0
code_type              0
format_group          35
format_subgroup       86
category_group        36
category_subgroup    533
age_group             35
dtype: int64

Again, with the size of the eventual DataFrame in mind, I want to drop any unnecessary columns before merging, so I'll drop the following columns:
- `code_type`, since that is superfluous information
- `category_subgroup`, since that is mostly NaN values

In [13]:
# drop columns
dd.drop(columns=['code_type', 'category_subgroup'], inplace=True)

In [14]:
%%time

# merge checkouts dataframe with info from data dictionary
df_merged = df.merge(dd, left_on='collection', right_on='code')

# take a look
df_merged.head()

CPU times: user 5.47 s, sys: 303 ms, total: 5.77 s
Wall time: 5.77 s


,collection,call_number,title,subjects,date,code,description,format_group,format_subgroup,category_group,age_group
0,nadvd,DVD FIREWAL,Firewall,"Kidnapping Drama, Video recordings for the hea...",2008-02-13,nadvd,"NA-DVD, Fiction",Media,Video Disc,Fiction,Adult
1,nadvd,DVD MARLEY,Marley me,"Comedy films, Married people Drama, Philadelph...",2009-07-03,nadvd,"NA-DVD, Fiction",Media,Video Disc,Fiction,Adult
2,nadvd,DVD SIX FEE Season 4,Six feet under The complete fourth season,"Video recordings for the hearing impaired, Pro...",2008-10-26,nadvd,"NA-DVD, Fiction",Media,Video Disc,Fiction,Adult
3,nadvd,DVD DOCTOR,Doctor Who The next doctor,"London England Drama, Doctor Who Fictitious ch...",2010-11-10,nadvd,"NA-DVD, Fiction",Media,Video Disc,Fiction,Adult
4,nadvd,DVD SCHOOL,School ties,"Antisemitism Drama, Video recordings for the h...",2008-12-28,nadvd,"NA-DVD, Fiction",Media,Video Disc,Fiction,Adult


After merging, I can now drop the `collection` and `code` columns, since those are no longer necessary.

In [15]:
# drop columns
df_merged.drop(columns=['collection', 'code'], inplace=True)

### Set `date` column as index

In [16]:
%%time

# set `date` column as index, sort by index, and drop it outside the index
# note: in the past, the dropping of the column was done by default, but that no longer seems to be the case?
df_merged = df_merged.set_index(df_merged.date).sort_index().drop(columns='date')

# take a look
df_merged.head()

CPU times: user 45.9 s, sys: 719 ms, total: 46.6 s
Wall time: 46.6 s


,call_number,title,subjects,description,format_group,format_subgroup,category_group,age_group
date,,,,,,,,
2008-01-02,E GRAHN,Whats going on in there,"Visual perception, Cities and towns Fiction",NC--Children's Picture Books,Print,Book,Fiction,Juvenile
2008-01-02,DVD J CAT FRO,cat from outer space,"Video recordings for the hearing impaired, Com...",NC-Children's DVDs,Media,Video Disc,Fiction,Juvenile
2008-01-02,DVD KUNDUN,Kundun,Bstan dzin rgya mtsho Dalai Lama XIV 1935 Dram...,"NA-DVD, Fiction",Media,Video Disc,Fiction,Adult
2008-01-02,E AHLBERG,shopping expedition,"Imagination Fiction, Shopping Fiction",NC--Children's Picture Books,Print,Book,Fiction,Juvenile
2008-01-02,DVD 332.024 C833 2003,courage to be rich creating a life of material...,"Wealth Moral and ethical aspects, Video record...","NA-DVD, Nonfiction",Media,Video Disc,Nonfiction,Adult


#### NOTE: I may be able to drop even more columns (thinking especially of `call_number`, `title`, and `subjects`), since I'll mostly be looking at sheer numbers of items checked out each day. I'll keep them in for now in case they end up being useful for EDA.

### 💾 Save/Load

In [17]:
%%time

# uncomment to save
with gzip.open('data/seattle_lib_temp_ten_mil.pkl', 'wb') as goodbye:
    pickle.dump(df_merged, goodbye, protocol=pickle.HIGHEST_PROTOCOL)
    
# # uncomment to load
# with gzip.open('data/seattle_lib_temp_ten_mil.pkl', 'rb') as hello:
#     df = pickle.load(hello)

CPU times: user 4min 19s, sys: 6.67 s, total: 4min 26s
Wall time: 4min 27s


#### NOTE: File size for 1 million rows is just under 38 MB; takes 25 seconds to save.

#### NOTE: File size for 10 million rows is just under 405 MB; takes 4.5 minutes to save.

## GRAVEYARD

In [18]:
please break code

SyntaxError: invalid syntax (<ipython-input-18-b8306b2d38fe>, line 1)

In [ ]:
dd[dd.code_type == 'ItemType']

In [ ]:
dd[dd.code == 'nadvd']

In [ ]:
dd[dd.code == 'acdvd']

In [ ]:
dd.code_type.unique()

In [ ]:
dd_item = dd[dd.code_type == 'ItemType'][['code', 'description', 'format_group', 'format_subgroup', 'category_group', 
             'category_subgroup', 'age_group']]

dd_item.head()

In [ ]:
dd_item2 = dd[dd.code_type == 'ItemCollection'][['code', 'description', 'format_group', 'format_subgroup', 'category_group', 
             'category_subgroup', 'age_group']]

dd_item2.head()

In [ ]:
sorted(df.item_type.unique())

In [ ]:
dd_loc = dd[dd.code_type == 'Location'][['code', 'description']]

dd_loc.head()

In [ ]:
test = df.merge(dd_item, left_on='item_type', right_on='code')
# test = test.merge(dd_loc, left_on='collection', right_on='code')

test.head()

In [ ]:
test.isna().sum()

In [ ]:
test.format_group.value_counts()

In [ ]:
test.collection.unique()

In [ ]:
test.shape

In [ ]:
test2 = df.merge(dd_item2, left_on='Collection', right_on='code')

test2.head()

In [ ]:
test.groupby('format_group').category_group.value_counts()

In [ ]:
test2.groupby('format_group').category_group.value_counts()

In [ ]:
dd[dd.code == 'nybot']

In [ ]:
sorted(df.collection.unique())

In [ ]:
dd_loc.code.unique()

In [ ]:
[cod for cod in df.collection.unique() if cod in dd_loc.code.unique()]